# Tensor Mathematics & Circuit Composition with *Julia Yao.jl / QML.jl*

**Aims**

1. Reproduce a Kronecker product: \(H \otimes I\).
2. Build a Bell state via `put!(H)` + `control` (CNOT) and measure.
3. Construct a controlled‑\(U\) (here \(R_y(\pi/4)\)) and inspect its matrix.
4. Create a Toffoli (CCNOT) gate and display its decomposition.

NB: Need to use the IJulia kernel.
    
    Tensor‑mathematics objectives using the Julia packages Yao.jl (core circuit library) and QML.jl (high‑level QML interface).

In [ ]:
# ╔═╡ 0000
# Cell 0 - Install / imports
using Pkg
Pkg.activate(temp=true)
Pkg.add(["Yao", "YaoPlots"])   # only first run

using Yao, YaoPlots, LinearAlgebra

## 1 Kronecker product: `H ⊗ I`

In [ ]:
# ╔═╡ 0001
H = 1/sqrt(2) * [1  1;
                 1 -1]

I2 = Matrix{Float64}(I, 2, 2)
kron_mat = kron(H, I2)

# Circuit representation
circuit = chain(2, put(1=>H))       # H on qubit 1, I on qubit 2
U = mat(circuit)

println("Equal matrices? ", isapprox(U, kron_mat))

## 2 Bell state via `H` + `control`

In [ ]:
# ╔═╡ 0002
bell = chain(2, put(1=>H), control(1, 2=>X))   # H(1) then CNOT
bell_state = zero_state(2) |> bell

display(bell_state)         # amplitude list
barplot(probabilities(bell_state); legend=false,
        title="Bell-state probabilities (|00⟩ + |11⟩)")

## 3 Controlled-$U$: $U = R_y(\pi/4)$

In [ ]:
# ╔═╡ 0003
theta = π/4
Ry = cos(theta/2)*I2 - 1im*sin(theta/2)*[0 -1; 1 0]   # matrix form

ctrl_Ry = control(1, 2=>matblock(Ry))
display(mat(ctrl_Ry))

## 4 Toffoli gate and decomposition

# ╔═╡ 0004
toffoli = control( (1,2), 3=>X )   # 1 & 2 controls → X on 3
println("Ideal Toffoli depth  :", depth(toffoli))

# Automatic decomposition into ElementaryBlock
decomp = decompose(toffoli)
plot(decomp; flatten=true, title="Toffoli decomposition")

## 5 Observations

* `kron` in Julia / LinearAlgebra reproduces the multi‑qubit unitary
  of independent gates.
* `control` extends any single‑qubit block to a controlled operation.
* Yao’s `decompose` reveals how three‑qubit gates reduce to one‑ and
  two‑qubit primitives, underscoring scalability from small building
  blocks to larger registers.

### Suggested extensions

1. Verify \((H \otimes H)\,\mathrm{CNOT}\,(H \otimes H)\) implements a CZ gate.
2. Build a four‑qubit GHZ state and confirm probabilities for `0000`
   and `1111`.
3. Swap the default CPU backend for the GPU backend (`CuYao`) to explore
   performance differences.